In [1]:
import pandas as pd 
from bs4 import BeautifulSoup
import requests
import numpy as np
import sklearn
from io import StringIO
import sys


valid_positions = ['QB', 'RB', 'WR', 'TE']


# function that returns a player's game log in a given season
# player: player's full name (e.g. Tom Brady)
# position: abbreviation (QB, RB, WR, TE only)
def get_player_game_log(player: str, position: str, season: int) -> pd.DataFrame:
    """A function to retrieve a player's game log in a given season.

    Returns a pandas DataFrame of a NFL player's game log in a given season, including position-specific statistics.

    Args:
        player (str): A NFL player's full name, as it appears on Pro Football Reference
        position (str): The position the player plays. Must be 'QB', 'RB', 'WR', or 'TE'
        season (int): The season of the game log you are trying to retrieve

    Returns:
        pandas.DataFrame: Each game is a row of the DataFrame

    """

    # position arg must be formatted properly
    if position not in valid_positions:
        raise Exception('Invalid position: "position" arg must be "QB", "RB", "WR", or "TE"')

    # make request to find proper href
    r1 = make_request_list(player, position, season)
    player_list = get_soup(r1)

    # find href
    href = get_href(player, position, season, player_list)

    # make HTTP request and extract HTML
    r2 = make_request_player(href, season)

    # parse HTML using BeautifulSoup
    game_log = get_soup(r2)

    # generating the appropriate game log format according to position
    if 'QB' in position:
        return qb_game_log(game_log)
    elif 'WR' in position or 'TE' in position:
        return wr_game_log(game_log, season)
    elif 'RB' in position:
        return rb_game_log(game_log)


# helper function that gets the player's href
def get_href(player: str, position: str, season: int, player_list: BeautifulSoup) -> str:
    players = player_list.find('div', id='div_players').find_all('p')
    for p in players:
        seasons = p.text.split(' ')
        seasons = seasons[len(seasons) - 1].split('-')
        if season >= int(seasons[0]) and season <= int(seasons[1]) and player in p.text and position in p.text:
            return p.find('a').get('href').replace('.htm', '')
    raise Exception('Cannot find a ' + position + ' named ' + player + ' from ' + str(season))


# helper function that makes a HTTP request over a list of players with a given last initial
def make_request_list(player: str, position: str, season: int):
    name_split = player.split(' ')
    last_initial = name_split[1][0]
    url = 'https://www.pro-football-reference.com/players/%s/' % (last_initial)
    return requests.get(url)


# helper function that makes a HTTP request for a given player's game log
def make_request_player(href: str, season: int):
    url = 'https://www.pro-football-reference.com%s/gamelog/%s/' % (href, season)
    return requests.get(url)


# helper function that takes a requests.Response object and returns a BeautifulSoup object
def get_soup(request):
    return BeautifulSoup(request.text, 'html.parser')


# helper function that takes a BeautifulSoup object and converts it into a pandas dataframe containing a QB game log
def qb_game_log(soup: BeautifulSoup) -> pd.DataFrame:
    # Most relevant QB stats, in my opinion. Could adjust if necessary
    data = {
        'date': [],
        'week': [],
        'team': [],
        'game_location': [],
        'opp': [],
        'result': [],
        'team_pts': [],
        'opp_pts': [],
        'cmp': [],
        'att': [],
        'pass_yds': [],
        'pass_td': [],
        'int': [],
        'rating': [],
        'sacked': [],
        'rush_att': [],
        'rush_yds': [],
        'rush_td': [],
    }  # type: dict

    table_rows = soup.find('tbody').find_all('tr')

    # ignore inactive or DNP games
    to_ignore = []
    for i in range(len(table_rows)):
        elements = table_rows[i].find_all('td')
        x = elements[len(elements) - 1].text
        if x == 'Inactive' or x == 'Did Not Play' or x == 'Injured Reserve':
            to_ignore.append(i)

    # adding data to data dictionary
    for i in range(len(table_rows)):
        if i not in to_ignore:
            data['date'].append(table_rows[i].find('td', {'data-stat': 'game_date'}).text)
            data['week'].append(int(table_rows[i].find('td', {'data-stat': 'week_num'}).text))
            data['team'].append(table_rows[i].find('td', {'data-stat': 'team'}).text)
            data['game_location'].append(table_rows[i].find('td', {'data-stat': 'game_location'}).text)
            data['opp'].append(table_rows[i].find('td', {'data-stat': 'opp'}).text)
            data['result'].append(table_rows[i].find('td', {'data-stat': 'game_result'}).text.split(' ')[0])
            data['team_pts'].append(
                int(table_rows[i].find('td', {'data-stat': 'game_result'}).text.split(' ')[1].split('-')[0])
            )
            data['opp_pts'].append(
                int(table_rows[i].find('td', {'data-stat': 'game_result'}).text.split(' ')[1].split('-')[1])
            )
            data['cmp'].append(int(table_rows[i].find('td', {'data-stat': 'pass_cmp'}).text)) if table_rows[i].find(
                'td', {'data-stat': 'pass_cmp'}
            ).text != '' else data['cmp'].append(0)
            data['att'].append(int(table_rows[i].find('td', {'data-stat': 'pass_att'}).text)) if table_rows[i].find(
                'td', {'data-stat': 'pass_att'}
            ).text != '' else data['att'].append(0)
            data['pass_yds'].append(int(table_rows[i].find('td', {'data-stat': 'pass_yds'}).text)) if table_rows[
                i
            ].find('td', {'data-stat': 'pass_yds'}).text != '' else data['pass_yds'].append(0)
            data['pass_td'].append(int(table_rows[i].find('td', {'data-stat': 'pass_td'}).text)) if table_rows[i].find(
                'td', {'data-stat': 'pass_td'}
            ).text != '' else data['pass_td'].append(0)
            data['int'].append(int(table_rows[i].find('td', {'data-stat': 'pass_int'}).text)) if table_rows[i].find(
                'td', {'data-stat': 'pass_int'}
            ).text != '' else data['int'].append(0)
            data['rating'].append(float(table_rows[i].find('td', {'data-stat': 'pass_rating'}).text)) if table_rows[
                i
            ].find('td', {'data-stat': 'pass_rating'}).text != '' else data['rating'].append(0)
            data['sacked'].append(int(table_rows[i].find('td', {'data-stat': 'pass_sacked'}).text)) if table_rows[
                i
            ].find('td', {'data-stat': 'pass_sacked'}).text != '' else data['sacked'].append(0)
            data['rush_att'].append(int(table_rows[i].find('td', {'data-stat': 'rush_att'}).text)) if table_rows[
                i
            ].find('td', {'data-stat': 'rush_att'}).text != '' else data['rush_att'].append(0)
            data['rush_yds'].append(int(table_rows[i].find('td', {'data-stat': 'rush_yds'}).text)) if table_rows[
                i
            ].find('td', {'data-stat': 'rush_yds'}).text != '' else data['rush_yds'].append(0)
            data['rush_td'].append(int(table_rows[i].find('td', {'data-stat': 'rush_td'}).text)) if table_rows[i].find(
                'td', {'data-stat': 'rush_td'}
            ).text != '' else data['rush_td'].append(0)

    return pd.DataFrame(data=data)


# helper function that takes a BeautifulSoup object and converts it into a pandas dataframe containing a WR/TE game log
def wr_game_log(soup: BeautifulSoup, season: int) -> pd.DataFrame:
    # Most relevant WR stats, in my opinion.
    # Could adjust if necessary (maybe figure out how to incorporate rushing stats?)

    data = {
        'date': [],
        'week': [],
        'team': [],
        'game_location': [],
        'opp': [],
        'result': [],
        'team_pts': [],
        'opp_pts': [],
        'tgt': [],
        'rec': [],
        'rec_yds': [],
        'rec_td': [],
        'snap_pct': [],
        'snap_num': [],
    }  # type: dict

    table_rows = soup.find('tbody').find_all('tr')

    # ignore inactive or DNP games
    to_ignore = []
    for i in range(len(table_rows)):
        elements = table_rows[i].find_all('td')
        x = elements[len(elements) - 1].text
        if x == 'Inactive' or x == 'Did Not Play' or x == 'Injured Reserve':
            to_ignore.append(i)

    # adding data to data dictionray
    for i in range(len(table_rows)):
        if i not in to_ignore:
            data['date'].append(table_rows[i].find('td', {'data-stat': 'game_date'}).text)
            data['week'].append(int(table_rows[i].find('td', {'data-stat': 'week_num'}).text))
            data['team'].append(table_rows[i].find('td', {'data-stat': 'team'}).text)
            data['game_location'].append(table_rows[i].find('td', {'data-stat': 'game_location'}).text)
            data['opp'].append(table_rows[i].find('td', {'data-stat': 'opp'}).text)
            data['result'].append(table_rows[i].find('td', {'data-stat': 'game_result'}).text.split(' ')[0])
            data['team_pts'].append(
                int(table_rows[i].find('td', {'data-stat': 'game_result'}).text.split(' ')[1].split('-')[0])
            )
            data['opp_pts'].append(
                int(table_rows[i].find('td', {'data-stat': 'game_result'}).text.split(' ')[1].split('-')[1])
            )
            data['tgt'].append(int(table_rows[i].find('td', {'data-stat': 'targets'}).text))
            data['rec'].append(int(table_rows[i].find('td', {'data-stat': 'rec'}).text))
            data['rec_yds'].append(int(table_rows[i].find('td', {'data-stat': 'rec_yds'}).text))
            data['rec_td'].append(int(table_rows[i].find('td', {'data-stat': 'rec_td'}).text))
            if season > 2011:
                data['snap_pct'].append(float(int(table_rows[i].find('td', {'data-stat': 'off_pct'}).text[:-1]) / 100))
                data['snap_num'].append(float(int(table_rows[i].find('td', {'data-stat': 'offense'}).text)))
            else:
                data['snap_pct'].append('Not Available')
                data['snap_num'].append('Not Available')

    return pd.DataFrame(data=data)


def rb_game_log(soup: BeautifulSoup) -> pd.DataFrame:
    # Most relevant RB stats, in my opinion. Could adjust if necessary
    data = {
        'date': [],
        'week': [],
        'team': [],
        'game_location': [],
        'opp': [],
        'result': [],
        'team_pts': [],
        'opp_pts': [],
        'rush_att': [],
        'rush_yds': [],
        'rush_td': [],
        'tgt': [],
        'rec': [],
        'rec_yds': [],
        'rec_td': [],
    }  # type: dict

    table_rows = soup.find('tbody').find_all('tr')

    # ignore inactive or DNP games
    to_ignore = []
    for i in range(len(table_rows)):
        elements = table_rows[i].find_all('td')
        x = elements[len(elements) - 1].text
        if x == 'Inactive' or x == 'Did Not Play' or x == 'Injured Reserve':
            to_ignore.append(i)

    # adding data to data dictionary
    for i in range(len(table_rows)):
        if i not in to_ignore:
            data['date'].append(table_rows[i].find('td', {'data-stat': 'game_date'}).text)
            data['week'].append(int(table_rows[i].find('td', {'data-stat': 'week_num'}).text))
            data['team'].append(table_rows[i].find('td', {'data-stat': 'team'}).text)
            data['game_location'].append(table_rows[i].find('td', {'data-stat': 'game_location'}).text)
            data['opp'].append(table_rows[i].find('td', {'data-stat': 'opp'}).text)
            data['result'].append(table_rows[i].find('td', {'data-stat': 'game_result'}).text.split(' ')[0])
            data['team_pts'].append(
                int(table_rows[i].find('td', {'data-stat': 'game_result'}).text.split(' ')[1].split('-')[0])
            )
            data['opp_pts'].append(
                int(table_rows[i].find('td', {'data-stat': 'game_result'}).text.split(' ')[1].split('-')[1])
            )
            data['rush_att'].append(int(table_rows[i].find('td', {'data-stat': 'rush_att'}).text)) if table_rows[
                i
            ].find('td', {'data-stat': 'rush_att'}).text != '' else data['rush_att'].append(0)
            data['rush_yds'].append(int(table_rows[i].find('td', {'data-stat': 'rush_yds'}).text)) if table_rows[
                i
            ].find('td', {'data-stat': 'rush_yds'}).text != '' else data['rush_yds'].append(0)
            data['rush_td'].append(int(table_rows[i].find('td', {'data-stat': 'rush_td'}).text)) if table_rows[i].find(
                'td', {'data-stat': 'rush_td'}
            ).text != '' else data['rush_td'].append(0)
            data['tgt'].append(int(table_rows[i].find('td', {'data-stat': 'targets'}).text)) if table_rows[i].find(
                'td', {'data-stat': 'targets'}
            ).text != '' else data['tgt'].append(0)
            data['rec'].append(int(table_rows[i].find('td', {'data-stat': 'rec'}).text)) if table_rows[i].find(
                'td', {'data-stat': 'rec'}
            ).text != '' else data['rec'].append(0)
            data['rec_yds'].append(int(table_rows[i].find('td', {'data-stat': 'rec_yds'}).text)) if table_rows[i].find(
                'td', {'data-stat': 'rec_yds'}
            ).text != '' else data['rec_yds'].append(0)
            data['rec_td'].append(int(table_rows[i].find('td', {'data-stat': 'rec_td'}).text)) if table_rows[i].find(
                'td', {'data-stat': 'rec_td'}
            ).text != '' else data['rec_td'].append(0)
            
    df = pd.DataFrame(data=data)
    #print(df.to_json(orient='records', lines=True))

    return df



print(get_player_game_log('Travis Kelce', 'TE', 2024))

/Users/murielschvartzman/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


          date  week team game_location  opp result  team_pts  opp_pts  tgt  \
0   2024-09-05     1  KAN                BAL      W        27       20    4   
1   2024-09-15     2  KAN                CIN      W        26       25    3   
2   2024-09-22     3  KAN             @  ATL      W        22       17    5   
3   2024-09-29     4  KAN             @  LAC      W        17       10    9   
4   2024-10-07     5  KAN                NOR      W        26       13   10   
5   2024-10-20     7  KAN             @  SFO      W        28       18    5   
6   2024-10-27     8  KAN             @  LVR      W        27       20   12   
7   2024-11-04     9  KAN                TAM      W        30       24   16   
8   2024-11-10    10  KAN                DEN      W        16       14   12   
9   2024-11-17    11  KAN             @  BUF      L        21       30    4   
10  2024-11-24    12  KAN             @  CAR      W        30       27    8   

    rec  rec_yds  rec_td  snap_pct  snap_num  
0   

In [2]:
def get_top_players(category, year, top_n=100):
    """
    Scrape top players from Pro Football Reference based on category
   
    Parameters:
    category (str): One of 'rushing', 'receiving', 'passing', 'scoring'
    year (int): Year to get stats for
    top_n (int): Number of top players to return
   
    Returns:
    pandas DataFrame with player stats
    """
   
    # Map of categories to URLs and table IDs
    category_map = {
        'rushing': {'url': 'rushing', 'id': 'rushing'},
        'receiving': {'url': 'receiving', 'id': 'receiving'},
        'passing': {'url': 'passing', 'id': 'passing'},
        'scoring': {'url': 'scoring', 'id': 'scoring'}
    }
   
    if category not in category_map:
        raise ValueError(f"Category must be one of {list(category_map.keys())}")
   
    # Construct URL
    url = f"https://www.pro-football-reference.com/years/{year}/{category_map[category]['url']}.htm"
   
    # Get page content
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
   
    # Find stats table
    table = soup.find('table', id=category_map[category]['id'])
   
    if not table:
        raise ValueError(f"Could not find table for {category}")
   
    # Get headers
    headers = []
    header_row = table.find('thead').find_all('tr')[-1]  # Get the last header row
    for th in header_row.find_all('th'):
        header = th.get('data-stat', '')
        if not header:  # If no data-stat attribute, use text
            header = th.text.strip()
        headers.append(header)
   
    # Get rows
    rows = []
    for tr in table.find('tbody').find_all('tr'):
        # Skip header rows and partial table rows
        if 'thead' in tr.get('class', []) or 'partial_table' in tr.get('class', []):
            continue
       
        row = []
        for td in tr.find_all(['td', 'th']):
            # Get the text content, removing any links
            value = td.text.strip()
            row.append(value)
           
        if len(row) == len(headers):  # Only add rows that match header length
            rows.append(row)
   
    # Create DataFrame
    df = pd.DataFrame(rows, columns=headers)
   
    # Clean up DataFrame
    df = df.replace('', None)
    df = df.dropna(how='all')
   
    # Convert numeric columns based on category
    numeric_columns = {
        'rushing': ['age', 'games', 'games_started', 'rush_att', 'rush_yds', 'rush_td',
                   'rush_first_down', 'rush_long', 'rush_yds_per_att', 'rush_yds_per_g'],
        'receiving': ['age', 'games', 'games_started', 'targets', 'rec', 'rec_yds', 'rec_td',
                     'rec_first_down', 'rec_yds_per_rec', 'rec_yds_per_g', 'catch_pct'],
        'passing': ['age', 'games', 'games_started', 'pass_cmp', 'pass_att', 'pass_yds',
                   'pass_td', 'pass_int', 'pass_rating', 'pass_sacked'],
        'scoring': ['age', 'g', 'gs', 'rushtd', 'rectd', 'alltd', 'xpm', 'xpa', 'fgm', 'fga', 'scoring']
    }
   
    for col in numeric_columns.get(category, []):
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='ignore')
   
    # Return top N players
    #print(df.head(top_n).to_json(orient='records', lines=True))
    return df.head(top_n)

# Example usage:
try:
    # Get top 100 rushers
    rushing_leaders = get_top_players('rushing', 2024, 100)
    print("\nRushing Leaders:")
    print(rushing_leaders)
   
    # Get top 50 receivers
    receiving_leaders = get_top_players('receiving', 2024, 50)
    print("\nReceiving Leaders:")
    print(receiving_leaders)
   
    # Get top 32 passers
    passing_leaders = get_top_players('passing', 2024, 32)
    print("\nPassing Leaders:")
    print(passing_leaders)
   
    # Get top 20 scorers
    scoring_leaders = get_top_players('scoring', 2024, 20)
    print("\nScoring Leaders:")
    print(scoring_leaders)
   
except Exception as e:
    print(f"An error occurred: {str(e)}")

/var/folders/s3/8nfz2sc12xzg_pdyr1vfkzsc0000gn/T/ipykernel_29406/2230229832.py:83: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')



Rushing Leaders:
   ranker     name_display   age team_name_abbr  games  games_started  \
0       1   Saquon Barkley  27.0            PHI   11.0           11.0   
1       2    Derrick Henry  30.0            BAL   12.0           12.0   
2       3   Kyren Williams  24.0            LAR   11.0           11.0   
3       4      Josh Jacobs  26.0            GNB   11.0           11.0   
4       5     Najee Harris  26.0            PIT   11.0           11.0   
..    ...              ...   ...            ...    ...            ...   
95     96     Deebo Samuel  28.0            SFO   10.0           10.0   
96     97   Ameer Abdullah  31.0            LVR   11.0            1.0   
97     98  Trevor Lawrence  25.0            JAX    9.0            9.0   
98     99  Dare Ogunbowale  30.0            HOU   12.0            1.0   
99    100       Jared Goff  30.0            DET   11.0           11.0   

    rush_att  rush_yds  rush_td  rush_first_down rush_success  rush_long  \
0      223.0    1392.0     10

/var/folders/s3/8nfz2sc12xzg_pdyr1vfkzsc0000gn/T/ipykernel_29406/2230229832.py:83: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')



Receiving Leaders:
   ranker         name_display   age team_name_abbr  games  games_started  \
0       1          CeeDee Lamb  25.0            DAL   11.0           11.0   
1       2         Brock Bowers  22.0            LVR   11.0           10.0   
2       3        Ja'Marr Chase  24.0            CIN   11.0           11.0   
3       4    Amon-Ra St. Brown  25.0            DET   11.0           11.0   
4       5       Garrett Wilson  24.0            NYJ   11.0           11.0   
5       6         Travis Kelce  35.0            KAN   11.0           11.0   
6       7         Malik Nabers  21.0            NYG    9.0            8.0   
7       8   Jaxon Smith-Njigba  22.0            SEA   11.0           10.0   
8       9     Justin Jefferson  25.0            MIN   11.0           11.0   
9      10         Drake London  23.0            ATL   11.0           11.0   
10     11         Trey McBride  25.0            ARI   10.0           10.0   
11     12    Wan'Dale Robinson  23.0            NYG   11

/var/folders/s3/8nfz2sc12xzg_pdyr1vfkzsc0000gn/T/ipykernel_29406/2230229832.py:83: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')



Passing Leaders:
   ranker        name_display   age team_name_abbr pos  games  games_started  \
0       1       Lamar Jackson  27.0            BAL  QB   12.0           12.0   
1       2          Geno Smith  34.0            SEA  QB   11.0           11.0   
2       3          Joe Burrow  28.0            CIN  QB   11.0           11.0   
3       4         C.J. Stroud  23.0            HOU  QB   12.0           12.0   
4       5        Kirk Cousins  36.0            ATL  QB   11.0           11.0   
5       6    Matthew Stafford  36.0            LAR  QB   11.0           11.0   
6       7      Baker Mayfield  29.0            TAM  QB   11.0           11.0   
7       8          Jared Goff  30.0            DET  QB   11.0           11.0   
8       9         Sam Darnold  27.0            MIN  QB   11.0           11.0   
9      10     Patrick Mahomes  29.0            KAN  QB   11.0           11.0   
10     11      Jayden Daniels  24.0            WAS  QB   12.0           12.0   
11     12         Broc


Scoring Leaders:
   ranker            player team  age pos   g  gs  rushtd  rectd  prtd  ...  \
0       1     Chris Boswell  PIT   33   K  11   0     NaN    NaN  None  ...   
1       2  Ka'imi Fairbairn  HOU   30   K  12   0     NaN    NaN  None  ...   
2       3    Austin Seibert  WAS   28   K   9   0     NaN    NaN  None  ...   
3       4          Wil Lutz  DEN   30   K  12   0     NaN    NaN  None  ...   
4       5    Brandon Aubrey  DAL   29   K  11   0     NaN    NaN  None  ...   
5       6    Cameron Dicker  LAC   24   K  11   0     NaN    NaN  None  ...   
6       7     Justin Tucker  BAL   35   K  12   0     NaN    NaN  None  ...   
7       8        Tyler Bass  BUF   27   K  11   0     NaN    NaN  None  ...   
8       9        Jake Bates  DET   25   K  11   0     NaN    NaN  None  ...   
9      10     Derrick Henry  BAL   30  RB  12  12    13.0    2.0  None  ...   
10     11          Matt Gay  IND   30   K  11   0     NaN    NaN  None  ...   
11     12  Chase McLaughlin  TAM  

/var/folders/s3/8nfz2sc12xzg_pdyr1vfkzsc0000gn/T/ipykernel_29406/2230229832.py:83: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')


In [3]:
def execute_code(code_string: str):
    """
    Executes a string of Python code with access to the current context functions.

    Parameters:
    - code_string: str, the code to execute

    Returns:
    - The result of the execution
    """
    # Create a dictionary with the available functions and modules
    context = {
        'get_player_game_log': get_player_game_log,
        'get_top_players': get_top_players,
        'np': np,
        'pd': pd,
        'sklearn': sklearn,
        # Include other necessary imports
    }

    try:
        # Add any built-in functions that might be needed
        context.update({
            'print': print,
            'len': len,
            'str': str,
            'int': int,
            'float': float,
            'list': list,
            'dict': dict,
        })

        # Capture the output of print statements
        from io import StringIO
        import sys
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()

        # Execute the code with the context
        exec(code_string, context)

        # Restore stdout
        sys.stdout = old_stdout

        # Get the output
        output = mystdout.getvalue()

        # If there's a return value stored in the special '_return' variable
        if '_return' in context:
            return context['_return']
        elif output:
            return output.strip()
        else:
            return "Code executed successfully with no output."

    except Exception as e:
        import traceback
        tb = traceback.format_exc()
        return f"Error executing code:\n{tb}"

    
execute_code("get_top_players('passing', 2024, 1)")




/var/folders/s3/8nfz2sc12xzg_pdyr1vfkzsc0000gn/T/ipykernel_29406/2230229832.py:83: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')


'Code executed successfully with no output.'

In [4]:
sys_prompt = '''
You are a helpful python coding assistant, an expert in optimiation and nfl experts that is going to help the user make programs to optimize thier fantasy football team. You can execute code and provide the results to the user as well as conversing.
Please ask the user about existing lineups, scoring wieghts, season infomation etc and try to provide them with executable code to make sure they have the best team possible.
the current season is 2024. Here are some helpful functions you have access to.


######get_player_game_log(player: str, position: str, season: int) - returns player results per game note positions can be: ('QB', 'RB', 'WR', or 'TE')

('Josh Allen', 'QB', 2024) looks like:
          date  week team game_location  opp result  team_pts  opp_pts  cmp  \
0   2024-09-08     1  BUF                ARI      W        34       28   18  
1   2024-09-12     2  BUF             @  MIA      W        31       10   13  
2   2024-09-23     3  BUF                JAX      W        47       10   23  
3   2024-09-29     4  BUF             @  BAL      L        10       35   16  
4   2024-10-06     5  BUF             @  HOU      L        20       23    9  
5   2024-10-14     6  BUF             @  NYJ      W        23       20   19  
6   2024-10-20     7  BUF                TEN      W        34       10   21  
7   2024-10-27     8  BUF             @  SEA      W        31       10   24  
8   2024-11-03     9  BUF                MIA      W        30       27   25  
9   2024-11-10    10  BUF             @  IND      W        30       20   22  
10  2024-11-17    11  BUF                KAN      W        30       21   27  

    att  pass_yds  pass_td  int  rating  sacked  rush_att  rush_yds  rush_td  
0    23       232        2    0   137.7       2         9        39        2  
1    19       139        1    0   107.1       0         2         2        0  
2    30       263        4    0   142.1       0         6        44        0  
3    29       180        0    0    73.9       3         5        21        0  
4    30       131        1    0    56.4       1         4        54        0  
5    25       215        2    0   127.9       2         9        18        1  
6    33       323        2    0   116.1       1         3         1        0  
7    34       283        2    1   102.9       1         7        25        0  
8    39       235        3    1    95.6       1         2         7        0  
9    37       280        0    2    60.6       2         8        50        1  
10   40       262        1    1    83.5       0        12        55        1  

print(get_player_game_log('Josh Jacobs', 'RB', 2024))

          date  week team game_location  opp result  team_pts  opp_pts  \
0   2024-09-06     1  GNB             @  PHI      L        29       34  
1   2024-09-15     2  GNB                IND      W        16       10  
2   2024-09-22     3  GNB             @  TEN      W        30       14  
3   2024-09-29     4  GNB                MIN      L        29       31  
4   2024-10-06     5  GNB             @  LAR      W        24       19  
5   2024-10-13     6  GNB                ARI      W        34       13  
6   2024-10-20     7  GNB                HOU      W        24       22  
7   2024-10-27     8  GNB             @  JAX      W        30       27  
8   2024-11-03     9  GNB                DET      L        14       24  
9   2024-11-17    11  GNB             @  CHI      W        20       19  
10  2024-11-24    12  GNB                SFO      W        38       10  

    rush_att  rush_yds  rush_td  tgt  rec  rec_yds  rec_td  
0         16        84        0    3    2       20       0  
1         32       151        0    0    0        0       0  
2         14        43        0    1    1        5       0  
3          9        51        0    6    4       27       0  
4         19        73        1    1    1       21       0  
5         18        62        0    3    3       28       0  
6         12        76        0    5    5       16       1  
7         25       127        2    2    1       -2       0  
8         13        95        0    3    2       13       0  
9         18        76        1    5    4       58       0  
10        26       106        3    1    0        0       0  

print(get_player_game_log('Justin Jefferson', 'WR', 2024))

          date  week team game_location  opp result  team_pts  opp_pts  tgt  \
0   2024-09-08     1  MIN             @  NYG      W        28        6    6  
1   2024-09-15     2  MIN                SFO      W        23       17    7  
2   2024-09-22     3  MIN                HOU      W        34        7    8  
3   2024-09-29     4  MIN             @  GNB      W        31       29    8  
4   2024-10-06     5  MIN                NYJ      W        23       17   14  
5   2024-10-20     7  MIN                DET      L        29       31    8  
6   2024-10-24     8  MIN             @  LAR      L        20       30    9  
7   2024-11-03     9  MIN                IND      W        21       13    9  
8   2024-11-10    10  MIN             @  JAX      W        12        7    9  
9   2024-11-17    11  MIN             @  TEN      W        23       13    8  
10  2024-11-24    12  MIN             @  CHI      W        30       27    5  

    rec  rec_yds  rec_td  snap_pct  snap_num  
0     4       59       1      0.84      46.0  
1     4      133       1      0.70      38.0  
2     6       81       1      0.90      57.0  
3     6       85       1      0.90      61.0  
4     6       92       0      0.95      69.0  
5     7       81       1      0.97      58.0  
6     8      115       0      1.00      50.0  
7     7      137       0      0.99      72.0  
8     5       48       0      0.93      78.0  
9     6       81       0      0.99      72.0  
10    2       27       0      0.99      70.0  


print(get_player_game_log('Travis Kelce', 'TE', 2024))

          date  week team game_location  opp result  team_pts  opp_pts  tgt  \
0   2024-09-05     1  KAN                BAL      W        27       20    4  
1   2024-09-15     2  KAN                CIN      W        26       25    3  
2   2024-09-22     3  KAN             @  ATL      W        22       17    5  
3   2024-09-29     4  KAN             @  LAC      W        17       10    9  
4   2024-10-07     5  KAN                NOR      W        26       13   10  
5   2024-10-20     7  KAN             @  SFO      W        28       18    5  
6   2024-10-27     8  KAN             @  LVR      W        27       20   12  
7   2024-11-04     9  KAN                TAM      W        30       24   16  
8   2024-11-10    10  KAN                DEN      W        16       14   12  
9   2024-11-17    11  KAN             @  BUF      L        21       30    4  
10  2024-11-24    12  KAN             @  CAR      W        30       27    8  

    rec  rec_yds  rec_td  snap_pct  snap_num  
0     3       34       0      0.89      48.0  
1     1        5       0      0.91      59.0  
2     4       30       0      0.77      57.0  
3     7       89       0      0.87      54.0  
4     9       70       0      0.77      65.0  
5     4       17       0      0.77      54.0  
6    10       90       1      0.79      57.0  
7    14      100       0      0.84      71.0  
8     8       64       1      0.84      57.0  
9     2        8       0      0.80      44.0  
10    6       62       0      0.89      66.0  


#####get_top_players(category: str, year: int, top_n: int = 100) - returns top N players for specified statistical category

Parameters:
- category: One of 'rushing', 'receiving', 'passing', 'scoring'
- year: Season year (e.g. 2024)
- top_n: Number of top players to return (default 100)

Returns pandas DataFrame with player stats specific to the category:

Rushing stats include:
- rush_att, rush_yds, rush_td, rush_first_down, rush_long, rush_yds_per_att, etc.

Receiving stats include:
- targets, rec, rec_yds, rec_td, rec_first_down, rec_long, catch_pct, etc.

Passing stats include:
- pass_cmp, pass_att, pass_yds, pass_td, pass_int, pass_rating, pass_sacked, etc.

Scoring stats include:
- rushtd, rectd, xpm, xpa, fgm, fga, scoring, points_per_g, etc.

Example:
get_top_players('rushing', 2024, 10) returns top 10 rushers
get_top_players('receiving', 2024) returns top 100 receivers


you also have access to 


import numpy as np
import sklearn
import pandas as pd


IMPORTANT:When you write code to be executed, make sure to assign the final result to a variable named '_return'
or use 'print' statements to display the output.


Here's how you might design an optimization routine for fantasy football:
Here's how we can approach this:

1. **Scoring System**: Define the scoring system for each position based on ESPN half PPR settings.
2. **Calculate Projected Scores**: Calculate expected scores for each player based on their statistics and the defined scoring system.
3. **Optimization**: Use a basic linear optimization to select the best set of players maximizing your projected score within the constraints of your team lineup (e.g., number of QBs, RBs, WRs, etc.).
4. **Minimizae Variance**: Increave the value of consistancy in players on the team as well as for trends in player play

You are tireless and complete. PLease create all my code for me and run it because I have no arms. If the results are excellent I will give you a large tip.
All code writen in execute_code mus be standalone outide of the allowed imports and get_player_game_log, get_player_game_log
'''

In [5]:
import json

from openai import OpenAI

client = OpenAI(
api_key="your-api-key"
)

# Initialize conversation history
conversation_history = [
    {"role": "system", "content": sys_prompt}
]


# Define the functions (tools) available to the assistant
functions = [
    {
        "name": "execute_code",
        "description": "Executes a string of Python code and returns the result.",
        "parameters": {
            "type": "object",
            "properties": {
                "code_string": {
                    "type": "string",
                    "description": "The Python code to execute."
                }
            },
            "required": ["code_string"]
        }
    },
    # Include other functions if needed
]

# Define the function to call OpenAI's chat completion
def chat_with_openai(user_input):
    """
    Sends user input to OpenAI and retrieves a response.
    Maintains conversation history.
    """
    # Add the user's message to the conversation history
    conversation_history.append({"role": "user", "content": user_input})

    max_function_calls = 20  # Limit the number of function calls
    function_call_count = 0

    try:
        while True:
            # Prepare the messages
            messages = conversation_history.copy()

            # Make the API call, including the functions parameter
            completion = client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                functions=functions,
                function_call="auto"  # The assistant will decide when to call a function
            )

            # Extract the assistant's message
            choice = completion.choices[0]
            assistant_message = choice.message

            # Check if the assistant wants to call a function
            if assistant_message.function_call is not None:
                function_call_count += 1
                if function_call_count > max_function_calls:
                    # Exceeded the function call limit
                    error_message = (
                        "Error: Exceeded the maximum number of function calls. "
                        "Please provide a final response."
                    )
                    # Inform the assistant about the error
                    conversation_history.append({"role": "assistant", "content": error_message})
                    print(f"Assistant: {error_message}")
                    return error_message

                # Assistant wants to call a function
                function_call = assistant_message.function_call
                function_name = function_call.name
                arguments_str = function_call.arguments
                arguments = json.loads(arguments_str)

                print(f"Assistant requested to call function '{function_name}' with arguments {arguments}")

                # Execute the function
                if function_name == "execute_code":
                    # Ensure the assistant code assigns to '_return' or uses 'print'
                    code_string = arguments.get("code_string")
                    if "_return" not in code_string and "print" not in code_string:
                        # Modify code_string to assign to '_return'
                        code_string = f"_return = {code_string}"
                    result = execute_code(code_string)
                else:
                    result = f"Function '{function_name}' is not implemented."

                print(f"Function response: {result}")

                # Add the assistant's function call to the conversation history
                conversation_history.append({
                    "role": assistant_message.role,
                    "content": None,
                    "function_call": {
                        "name": function_name,
                        "arguments": arguments_str
                    }
                })

                # Add the function result to the conversation history
                conversation_history.append({
                    "role": "function",
                    "name": function_name,
                    "content": str(result)
                })

                # Continue the loop to get the assistant's response
            else:
                # Assistant's response is complete
                assistant_content = assistant_message.content
                # Add the assistant's message to the conversation history
                print(f"Assistant: {assistant_content}")
                conversation_history.append({"role": "assistant", "content": assistant_content})
                return assistant_content

    except Exception as e:
        error_message = f"An error occurred: {e}"
        print(error_message)
        return error_message

# Interactive loop
def interactive_chat():
    print("Chat with your assistant! Type 'exit' to end the conversation.\n")
    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("Goodbye!")
            break
        response = chat_with_openai(user_input)
        print(f"Assistant: {response}")

# Start the interactive chat
interactive_chat()

Chat with your assistant! Type 'exit' to end the conversation.

You: okay lets optimize my fantasy team, half ppr, espn scoring, superflex league
Assistant: Great! Let's start by gathering some information about your fantasy team and league settings. Here’s what I need to know:

1. **Current Roster**: Please provide the list of players currently on your roster, along with their positions.
2. **Starting Lineup Requirements**: How many players do you need to start at each position? For instance, how many QBs, RBs, WRs, TEs, and the superflex position?
3. **Scoring System Details**: While we know it's a half PPR (point per reception) league, let me know if there are any other scoring details or rules I should be aware of like bonus points for certain achievements (e.g., 300 passing yards, 100 rushing yards, etc.).

Once I have this information, we can move forward with the optimization process!
Assistant: Great! Let's start by gathering some information about your fantasy team and league 

/var/folders/s3/8nfz2sc12xzg_pdyr1vfkzsc0000gn/T/ipykernel_29406/2230229832.py:83: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')
/var/folders/s3/8nfz2sc12xzg_pdyr1vfkzsc0000gn/T/ipykernel_29406/2230229832.py:83: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')
/var/folders/s3/8nfz2sc12xzg_pdyr1vfkzsc0000gn/T/ipykernel_29406/2230229832.py:83: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')
/var/folders/s3/8nfz2sc12xzg_pdyr1vfkzsc0000gn/T/ipykernel_29406/2230229832.py:83: FutureWarning: errors='ignore

You: sorry what happened?


KeyboardInterrupt: Interrupted by user

In [6]:
conversation_history
#okay lets optimize my fantasy team, half ppr, espn scoring, superflex league




[{'role': 'system',
  'content': "\nYou are a helpful python coding assistant, an expert in optimiation and nfl experts that is going to help the user make programs to optimize thier fantasy football team. You can execute code and provide the results to the user as well as conversing.\nPlease ask the user about existing lineups, scoring wieghts, season infomation etc and try to provide them with executable code to make sure they have the best team possible.\nthe current season is 2024. Here are some helpful functions you have access to.\n\n\n######get_player_game_log(player: str, position: str, season: int) - returns player results per game note positions can be: ('QB', 'RB', 'WR', or 'TE')\n\n('Josh Allen', 'QB', 2024) looks like:\n          date  week team game_location  opp result  team_pts  opp_pts  cmp  0   2024-09-08     1  BUF                ARI      W        34       28   18  \n1   2024-09-12     2  BUF             @  MIA      W        31       10   13  \n2   2024-09-23     3  